In [17]:
from tqdm import tqdm
import numpy as np

In [2]:
trainTextPath = '../texts_train.txt'
trainScorePath = '../scores_train.txt'

In [3]:
def loadTrain():
    with open(trainTextPath) as f, open(trainScorePath) as scores:
        return f.readlines(), scores.readlines()

In [4]:
X, y = loadTrain()

In [5]:
from pymystem3 import Mystem

m = Mystem()

def lemmatize(sentence):
    return m.lemmatize(sentence)

In [6]:
import warnings
warnings.filterwarnings(action='once')

In [7]:
X_lemmatized = []
for sentence in tqdm(X):
    X_lemmatized.append(lemmatize(sentence))

100%|██████████| 20000/20000 [00:35<00:00, 569.14it/s] 


In [8]:
import re

def check(token):
    return len(token) > 1

def filterTokens(sentence):
    return [token for token in sentence if check(token)]

In [9]:
from nltk.corpus import stopwords
import nltk

In [10]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vladimir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
filteredX = [filterTokens(tokens) for tokens in  X_lemmatized]

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idfVectorization(X):
    vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(1, 3), stop_words=stopwords.words("russian"))
    x_train = vectorizer.fit_transform([' '.join(x) for x in X])
    return vectorizer, x_train

In [15]:
vectrizer, vectorized = tf_idfVectorization(filteredX)

In [96]:
def test(model):
    pred = np.array([str(np.clip(int(round(model.predict(xi)[0])), 1, 10)) for xi in vectorized[16000:]])
    return np.mean(((np.array([int(pi) for pi in pred]) - np.array([int(yi) for yi in y[16000:]])) ** 2))

In [105]:
print(np.mean((np.array([int(yi) for yi in y]))))

7.8584


In [78]:
from sklearn import linear_model

reg = linear_model.LinearRegression().fit(vectorized[:-2000], [str(int(yi) + np.random.random() * 2 - 1) for yi in y[:-2000]])

In [79]:
test(reg)

3.6695

In [97]:
reg = linear_model.LinearRegression().fit(vectorized[:-4000], y[:-4000])

In [98]:
test(reg)

3.9485

In [101]:
ridge = linear_model.Ridge().fit(vectorized[:-2000], [str(int(yi) + np.random.random() * 2 - 1) for yi in y[:-2000]])

In [102]:
test(ridge)

2.4535

In [20]:
def processLine(line, model):
    x_test = vectrizer.transform([' '.join(filterTokens(lemmatize(line)))])
    return str(np.clip(int(round(model.predict(x_test)[0])), 1, 10))

In [23]:
testInPath = 'test.in.txt'
testOutPath = 'test.out'

In [18]:
def processTest(model):
    with open(testInPath, 'r') as input, open(testOutPath, 'w') as output:
            for line in tqdm(input.readlines()):
                output.write(model(line) + '\n')

In [103]:
import os

processTest(lambda sentence: processLine(sentence, ridge))

100%|██████████| 1000/1000 [00:03<00:00, 295.62it/s]
